# Full notebook
This notebook has the whole process put together:
1. Loading dataset
2. Defining grammar
3. Searching through the grammar and evaluating candidate pipelines

## 0. Imports

In [54]:
# uncomment the following if not all packages are added.

# import Pkg
# using Pkg
# Pkg.add("HTTP")
# Pkg.add("JSON")
# Pkg.add("DataFrames")
# Pkg.add("OpenML")
# Pkg.add("DataFrames") 
# Pkg.add("CSV") 
# Pkg.add("Suppressor")

In [55]:
using ScikitLearn
using ScikitLearn.Pipelines: Pipeline, FeatureUnion
using ScikitLearn.CrossValidation: cross_val_score
using XGBoost
using Revise
using Random
using Statistics: mean
using ExprRules: get_executable
using Suppressor

include("../Herb.jl/src/Herb.jl")
include("helper.jl")

get_class_type_dataset (generic function with 1 method)

In [56]:
# import the sk-learn functions
@sk_import decomposition: (PCA)
@sk_import preprocessing: (StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, Binarizer, PolynomialFeatures)
@sk_import feature_selection: (VarianceThreshold, SelectKBest, SelectPercentile, SelectFwe, RFE)
@sk_import tree: (DecisionTreeClassifier)
@sk_import ensemble: (RandomForestClassifier, GradientBoostingClassifier)
@sk_import linear_model: (LogisticRegression)
@sk_import neighbors: (NearestNeighbors)
@sk_import svm: (LinearSVC)

PyObject <class 'sklearn.svm._classes.LinearSVC'>

## 1. Loading datasets

In [57]:
# load dataset
dataset = get_dataset(1499)

# it does not work for the seeds table dataset!
# dataset = get_dataset(1499)

# does not work either on dataset 1464!

# shuffle the dataset
dataset_shuffled = dataset[shuffle(1:end), :]

# split into train and test sets (90:10)
split_index = floor(Int, size(dataset_shuffled, 1) * 0.9)
train_df = dataset_shuffled[1:split_index, :]
test_df = dataset_shuffled[split_index+1:end, :]

# show first five entries
first(dataset_shuffled, 5)

Row,V1,V2,V3,V4,V5,V6,V7,Class
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…
1,19.11,16.26,0.9081,6.154,3.93,2.936,6.079,2
2,11.27,12.97,0.8419,5.088,2.763,4.309,5.0,3
3,12.13,13.73,0.8081,5.394,2.745,4.825,5.22,3
4,13.89,14.02,0.888,5.439,3.199,3.986,4.738,1
5,10.91,12.8,0.8372,5.088,2.675,4.179,4.956,3


In [58]:
# show metadata
print("size: ", size(dataset_shuffled))
describe(dataset_shuffled)

size: (210, 8)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,V1,14.8475,10.59,14.355,21.18,0,Float64
2,V2,14.5593,12.41,14.32,17.25,0,Float64
3,V3,0.870999,0.8081,0.87345,0.9183,0,Float64
4,V4,5.62853,4.899,5.5235,6.675,0,Float64
5,V5,3.2586,2.63,3.237,4.033,0,Float64
6,V6,3.7002,0.7651,3.599,8.456,0,Float64
7,V7,5.40807,4.519,5.223,6.55,0,Float64
8,Class,,1,,3,0,"CategoricalValue{String, UInt32}"


In [59]:
# split into features and labels
train_X = train_df[:, 1:end-1]
train_Y = train_df[:, end]
test_X = test_df[:, 1:end-1]
test_Y = test_df[:, end]

# print ratio train/test
ratio = trunc(Int, 10.0 * (size(train_df)[1] / (size(train_df)[1] + size(test_df)[1])))
print("train:test ratio = ", ratio , ":", (10-ratio))

train:test ratio = 9:1

## 2. Defining grammar

In [60]:
grammar = Herb.HerbGrammar.@cfgrammar begin

    # this is the version with multiple classifiers possible
    # START   = CLASSIF | sequence(PRE, CLASSIF)
    # PRE     = PREPROC | FSELECT | sequence(PRE, PRE) | parallel(BRANCH, BRANCH)
    # BRANCH  = PRE | CLASSIF | sequence(PRE, CLASSIF) 

    # this is the version with only one classifier
    START   = Pipeline([CLASSIF]) | Pipeline([PRE, CLASSIF])
    PRE     = PREPROC | FSELECT | ("seq", Pipeline([PRE, PRE]))  | ("par", FeatureUnion([PRE, PRE])) 

    # preprocessing functions
    PREPROC =   
        ("StandardScaler", StandardScaler()) |
        ("RobustScaler", RobustScaler()) |
        ("MinMaxScaler", MinMaxScaler()) |
        ("MaxAbsScaler", MaxAbsScaler()) |
        ("PCA", PCA()) |
        ("Binarizer", Binarizer()) |
        ("PolynomialFeatures", PolynomialFeatures())

    # feature selection functions
    FSELECT =  
        ("VarianceThreshold", VarianceThreshold()) |
        ("SelectKBest",  SelectKBest()) |
        ("SelectPercentile",  SelectPercentile()) |
        ("SelectFwe",  SelectFwe()) |
        ("Recursive Feature Elimination",  RFE(LinearSVC())) 

    # classifiers
    CLASSIF =
        ("DecisionTree", DecisionTreeClassifier()) |
        ("RandomForest", RandomForestClassifier()) |
        ("Gradient Boosting Classifier", GradientBoostingClassifier()) |
        ("LogisticRegression", LogisticRegression()) #|
        # ("NearestNeighborClassifier", NearestNeighbors())
end
;

In [61]:
# print out all pipelines that can be assembled in two steps
cfe_print = Herb.HerbSearch.ContextFreeEnumerator(grammar, 2, :START)
for rule in cfe_print
    println(Herb.HerbSearch.rulenode2expr(rule, grammar))
end

Pipeline([("DecisionTree", DecisionTreeClassifier())])
Pipeline([("RandomForest", RandomForestClassifier())])
Pipeline([("Gradient Boosting Classifier", GradientBoostingClassifier())])
Pipeline([("LogisticRegression", LogisticRegression())])


## 3. Search

In [62]:
"""
Fits the pipeline to the training set and measures the accuracy on test set.
input:  pipeline, train_X, train_Y, test_X, test_Y
output: accuracy of pipeline
"""
function evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

    # # this gives the following warning often, so it is suppressed for now.
    # # ConvergenceWarning: lbfgs failed to converge
    # @suppress_err begin
        
    # fit the pipeline
    model = ScikitLearn.fit!(pipeline, Matrix(train_X), Array(train_Y))

    # make predictions
    predictions = ScikitLearn.predict(model, Matrix(test_X))

    # measure the accuracy
    accuracy = mean(predictions .== test_Y)
end

evaluate_pipeline

In [63]:
"""This function enumerates the grammar and finds the best pipeline. """
function find_best_pipeline_with_bfs_search(pipeline, train_X, train_Y, test_X, test_Y, search_depth)
    best_accuracy = 0.0
    best_pipeline = nothing

    enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, search_depth, :START)
    for rule in enumerator
        try
            # get pipeline and calculate accuracy
            pipeline = eval(Herb.HerbSearch.rulenode2expr(rule, grammar))
            accuracy = evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

            # update best pipeline
            if (accuracy > best_accuracy) 
                best_accuracy = accuracy
                best_pipeline = pipeline
            end
            
            # print accuracy of pipeline
            print("\n accuracy: ", round(accuracy, digits=2), " by ", string(pipeline))
        catch
            continue
        end
    end
    return (best_accuracy, best_pipeline)
end

find_best_pipeline_with_bfs_search

In [64]:
# find the best pipeline and accuracy with depth 2
(best_accuracy, best_pipeline) = find_best_pipeline_with_bfs_search(pipeline, train_X, train_Y, test_X, test_Y, 2)
println("\n\nBest pipeline: ", round(best_accuracy, digits=2))
print(best_pipeline)


 accuracy: 0.86 by Pipeline(Tuple{Any, Any}[("DecisionTree", PyObject DecisionTreeClassifier())], Any[PyObject DecisionTreeClassifier()])


 accuracy: 0.95 by 

Pipeline(Tuple{Any, Any}[("RandomForest", PyObject RandomForestClassifier())], Any[PyObject RandomForestClassifier()])
 accuracy: 0.95 by Pipeline(Tuple{Any, Any}[("Gradient Boosting Classifier", PyObject GradientBoostingClassifier())], Any[PyObject GradientBoostingClassifier()])
 accuracy: 0.9 by Pipeline(Tuple{Any, Any}[("LogisticRegression", PyObject LogisticRegression())], Any[PyObject LogisticRegression()])

Best pipeline: 0.95
Pipeline(Tuple{Any, Any}[("RandomForest", PyObject RandomForestClassifier())], Any[PyObject RandomForestClassifier()])

C:\Users\denys\.julia\conda\3\x86_64\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
